## Лабораторная работа №3
## Гранин М.М. вариант 7

Цель лабораторной работы: изучение способов подготовки выборки и подбора гиперпараметров на примере метода ближайших соседей.


**Задание:**
1. Выберите набор данных (датасет) для решения задачи классификации или регрессии.
2. В случае необходимости проведите удаление или заполнение пропусков и кодирование категориальных признаков.
3. С использованием метода train_test_split разделите выборку на обучающую и тестовую.
4. Обучите модель ближайших соседей для произвольно заданного гиперпараметра K. Оцените качество модели с помощью подходящих для задачи метрик.
5. Произведите подбор гиперпараметра K с использованием GridSearchCV и RandomizedSearchCV и кросс-валидации, оцените качество оптимальной модели. Используйте не менее двух стратегий кросс-валидации.
6. Сравните метрики качества исходной и оптимальной моделей.



In [4]:
#Датасет содержит данные о кредитах на покупку электроники, которые были одобрены Tinkoff.ru. 
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from warnings import simplefilter

simplefilter('ignore')

In [5]:
# записываем CSV-файл в объект DataFrame
data = pd.read_csv('./lab3.csv', encoding='cp1251', sep=';')

In [6]:
# смотрим на первые пять строк
data.head()

,client_id,gender,age,marital_status,job_position,credit_sum,credit_month,tariff_id,score_shk,education,living_region,monthly_income,credit_count,overdue_credit_count,open_account_flg
0,1,M,NaN,NaN,UMN,"59998,00",10,1.6,NaN,GRD,КРАСНОДАРСКИЙ КРАЙ,30000.0,1.0,1.0,0
1,2,F,NaN,MAR,UMN,"10889,00",6,1.1,NaN,NaN,МОСКВА,NaN,2.0,0.0,0
2,3,M,32.0,MAR,SPC,"10728,00",12,1.1,NaN,NaN,ОБЛ САРАТОВСКАЯ,NaN,5.0,0.0,0
3,4,F,27.0,NaN,SPC,"12009,09",12,1.1,NaN,NaN,ОБЛ ВОЛГОГРАДСКАЯ,NaN,2.0,0.0,0
4,5,M,45.0,NaN,SPC,NaN,10,1.1,"0,421385",SCH,ЧЕЛЯБИНСКАЯ ОБЛАСТЬ,NaN,1.0,0.0,0


## 1) Обработка пропусков в данных

In [7]:
#проверяем типы данных и заполненность столбцов
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 170746 entries, 0 to 170745
Data columns (total 15 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   client_id             170746 non-null  int64  
 1   gender                170746 non-null  object 
 2   age                   170743 non-null  float64
 3   marital_status        170743 non-null  object 
 4   job_position          170746 non-null  object 
 5   credit_sum            170744 non-null  object 
 6   credit_month          170746 non-null  int64  
 7   tariff_id             170746 non-null  float64
 8   score_shk             170739 non-null  object 
 9   education             170741 non-null  object 
 10  living_region         170554 non-null  object 
 11  monthly_income        170741 non-null  float64
 12  credit_count          161516 non-null  float64
 13  overdue_credit_count  161516 non-null  float64
 14  open_account_flg      170746 non-null  int64  
dtype

In [8]:
#удаляем столбец с номером клиента (так как он незначимый) 
# и с регионом проживания (так как он нуждается в серьезной предобработке)
data.drop(['client_id', 'living_region'], axis=1, inplace=True)

In [9]:
# анализируем столбец marital_status, смотрим, какое значение в нем является самым частым 
data['marital_status'].describe()

count     170743
unique         5
top          MAR
freq       93954
Name: marital_status, dtype: object

In [10]:
# анализируем столбец education, смотрим, какое в нем самое частое значение
data['education'].describe()

count     170741
unique         5
top          SCH
freq       87537
Name: education, dtype: object

In [11]:
# дозаполняем нечисловые столбцы с пропусками самыми часто встречающимися значениями
data['marital_status'].fillna('MAR', inplace=True)
data['education'].fillna('SCH', inplace=True)

In [12]:
# дозаполняем числовые столбцы с пропусками медианными значениями
data['age'].fillna(data['age'].median(), inplace=True)
data['credit_count'].fillna(data['credit_count'].median(), inplace=True)
data['overdue_credit_count'].fillna(data['overdue_credit_count'].median(), inplace=True)

In [13]:
#меняем в столбцах 'credit_sum', 'score_shk'  запятые на точки  и преобразуем их в числовой  формат
for i in ['credit_sum', 'score_shk']:
    data[i] = data[i].str.replace(',', '.').astype('float')

In [14]:
# дозаполняем ставшие теперь числовыми столбцы 'credit_sum', 'score_shk'   медианными значениями
data['score_shk'].fillna(data['score_shk'].median(), inplace=True)
data['monthly_income'].fillna(data['monthly_income'].median(), inplace=True)
data['credit_sum'].fillna(data['credit_sum'].median(), inplace=True)

In [15]:
# смотрим, что получилось
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 170746 entries, 0 to 170745
Data columns (total 13 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   gender                170746 non-null  object 
 1   age                   170746 non-null  float64
 2   marital_status        170746 non-null  object 
 3   job_position          170746 non-null  object 
 4   credit_sum            170746 non-null  float64
 5   credit_month          170746 non-null  int64  
 6   tariff_id             170746 non-null  float64
 7   score_shk             170746 non-null  float64
 8   education             170746 non-null  object 
 9   monthly_income        170746 non-null  float64
 10  credit_count          170746 non-null  float64
 11  overdue_credit_count  170746 non-null  float64
 12  open_account_flg      170746 non-null  int64  
dtypes: float64(7), int64(2), object(4)
memory usage: 16.9+ MB


## 2) Кодирование категориальных признаков

In [16]:
category_cols = ['gender', 'job_position', 'education', 'marital_status']

In [17]:
print("Количество уникальных значений\n")
for col in category_cols:
    print(f'{col}: {data[col].unique().size}')

Количество уникальных значений

gender: 2
job_position: 18
education: 5
marital_status: 5


In [18]:
# кодируем нечисловые столбцы методом дамми-кодирования
data = pd.concat([data, 
                      pd.get_dummies(data['gender'], prefix="gender"),
                      pd.get_dummies(data['job_position'], prefix="job_position"),
                      pd.get_dummies(data['education'], prefix="education"),
                      pd.get_dummies(data['marital_status'], prefix="marital_status")],
                     axis=1)

In [19]:
#удаляем старые нечисловые столбцы, вместо них уже появились новые числовые
data.drop(['gender','job_position','education','marital_status'], axis=1, inplace=True)

In [20]:
data.head()

,age,credit_sum,credit_month,tariff_id,score_shk,monthly_income,credit_count,overdue_credit_count,open_account_flg,gender_F,...,education_ACD,education_GRD,education_PGR,education_SCH,education_UGR,marital_status_CIV,marital_status_DIV,marital_status_MAR,marital_status_UNM,marital_status_WID
0,34.0,59998.00,10,1.6,0.461599,30000.0,1.0,1.0,0,False,...,False,True,False,False,False,False,False,True,False,False
1,34.0,10889.00,6,1.1,0.461599,35000.0,2.0,0.0,0,True,...,False,False,False,True,False,False,False,True,False,False
2,32.0,10728.00,12,1.1,0.461599,35000.0,5.0,0.0,0,False,...,False,False,False,True,False,False,False,True,False,False
3,27.0,12009.09,12,1.1,0.461599,35000.0,2.0,0.0,0,True,...,False,False,False,True,False,False,False,True,False,False
4,45.0,21229.00,10,1.1,0.421385,35000.0,1.0,0.0,0,False,...,False,False,False,True,False,False,False,True,False,False


## 3) Разделение выборки на обучающую и тестовую

In [21]:
data_sample = data.sample(n=20000)
y = data_sample['open_account_flg']
X = data_sample.drop('open_account_flg', axis=1)
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=21)

## 4) Масштабирование данных

In [22]:
scaler = MinMaxScaler().fit(x_train)
x_train = pd.DataFrame(scaler.transform(x_train), columns=x_train.columns)
x_test = pd.DataFrame(scaler.transform(x_test), columns=x_train.columns)
x_train.describe()

,age,credit_sum,credit_month,tariff_id,score_shk,monthly_income,credit_count,overdue_credit_count,gender_F,gender_M,...,education_ACD,education_GRD,education_PGR,education_SCH,education_UGR,marital_status_CIV,marital_status_DIV,marital_status_MAR,marital_status_UNM,marital_status_WID
count,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,...,10000.000000,10000.000000,10000.000000,10000.000000,10000.00000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000
mean,0.356927,0.187018,0.241673,0.344043,0.432088,0.087977,0.112179,0.021050,0.517900,0.482100,...,0.000400,0.427800,0.002600,0.510000,0.05920,0.020700,0.100900,0.551800,0.305700,0.020900
std,0.202510,0.130729,0.108326,0.251920,0.143069,0.062828,0.092797,0.101158,0.499704,0.499704,...,0.019997,0.494784,0.050926,0.499925,0.23601,0.142385,0.301211,0.497334,0.460726,0.143057
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.192308,0.095968,0.212121,0.106383,0.326975,0.048223,0.052632,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.326923,0.148114,0.212121,0.340426,0.425177,0.073604,0.105263,0.000000,1.000000,0.000000,...,0.000000,0.000000,0.000000,1.000000,0.00000,0.000000,0.000000,1.000000,0.000000,0.000000
75%,0.480769,0.234938,0.272727,0.638298,0.529240,0.111675,0.157895,0.000000,1.000000,1.000000,...,0.000000,1.000000,0.000000,1.000000,0.00000,0.000000,0.000000,1.000000,1.000000,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.00000,1.000000,1.000000,1.000000,1.000000,1.000000


## 5) Обучение KNN с произвольным k

In [23]:
from sklearn.metrics import mean_squared_error, r2_score

def print_metrics(y_test, y_pred):
    print(f"Среднее квадратичное отклонение: {mean_squared_error(y_test, y_pred, squared=False)}")
    print(f"Коэффициент детерминации: {r2_score(y_test, y_pred)}")
    
def print_cv_result(cv_model, x_test, y_test):
    print(f'Оптимизация метрики {cv_model.scoring}: {cv_model.best_score_}')
    print(f'Лучший параметр: {cv_model.best_params_}')
    print('Метрики на тестовом наборе')
    print_metrics(y_test, cv_model.predict(x_test))
    print()

In [24]:
base_k = 10
base_knn = KNeighborsClassifier(n_neighbors=base_k)
base_knn.fit(x_train, y_train)
y_pred_base = base_knn.predict(x_test)

In [25]:
print(f'Test metrics for KNN with k={base_k}\n')
print_metrics(y_test, y_pred_base)

Test metrics for KNN with k=10

Среднее квадратичное отклонение: 0.41988093550434036
Коэффициент детерминации: -0.23508034502386277


## 6) Кросс-валидация

In [26]:
metrics = ['accuracy', 'recall', 'f1']
cv_values = [5, 10]

for cv in cv_values:
    print(f'Результаты кросс-валидации при cv={cv}\n')
    for metric in metrics:    
        params = {'n_neighbors': range(1, 40)}
        knn_cv = RandomizedSearchCV(KNeighborsClassifier(), params, cv=cv, scoring=metric, n_jobs=-1)
        #knn_cv = GridSearchCV(KNeighborsClassifier(), params, cv=cv, scoring=metric, n_jobs=-1)
        knn_cv.fit(x_train, y_train)
        print_cv_result(knn_cv, x_test, y_test)

Результаты кросс-валидации при cv=5

Оптимизация метрики accuracy: 0.8213000000000001
Лучший параметр: {'n_neighbors': 33}
Метрики на тестовом наборе
Среднее квадратичное отклонение: 0.415571895103603
Коэффициент детерминации: -0.20986032663426601

Оптимизация метрики recall: 0.08096201428549198
Лучший параметр: {'n_neighbors': 7}
Метрики на тестовом наборе
Среднее квадратичное отклонение: 0.43162483709814475
Коэффициент детерминации: -0.30513595166163143

Оптимизация метрики f1: 0.13280417290504423
Лучший параметр: {'n_neighbors': 7}
Метрики на тестовом наборе
Среднее квадратичное отклонение: 0.43162483709814475
Коэффициент детерминации: -0.30513595166163143

Результаты кросс-валидации при cv=10

Оптимизация метрики accuracy: 0.8209
Лучший параметр: {'n_neighbors': 20}
Метрики на тестовом наборе
Среднее квадратичное отклонение: 0.4167733196834941
Коэффициент детерминации: -0.2168658872980429

Оптимизация метрики recall: 0.2507014276846679
Лучший параметр: {'n_neighbors': 1}
Метрики на

In [27]:
for cv in cv_values:
    print(f'Результаты кросс-валидации при cv={cv}\n')
    for metric in metrics:    
        params = {'n_neighbors': range(1, 40)}
        #knn_cv = RandomizedSearchCV(KNeighborsClassifier(), params, cv=cv, scoring=metric, n_jobs=-1)
        knn_cv = GridSearchCV(KNeighborsClassifier(), params, cv=cv, scoring=metric, n_jobs=-1)
        knn_cv.fit(x_train, y_train)
        print_cv_result(knn_cv, x_test, y_test)

Результаты кросс-валидации при cv=5

Оптимизация метрики accuracy: 0.8213000000000001
Лучший параметр: {'n_neighbors': 33}
Метрики на тестовом наборе
Среднее квадратичное отклонение: 0.415571895103603
Коэффициент детерминации: -0.20986032663426601

Оптимизация метрики recall: 0.25012215807410404
Лучший параметр: {'n_neighbors': 1}
Метрики на тестовом наборе
Среднее квадратичное отклонение: 0.5047771785649585
Коэффициент детерминации: -0.7850168571303473

Оптимизация метрики f1: 0.2542442728118995
Лучший параметр: {'n_neighbors': 1}
Метрики на тестовом наборе
Среднее квадратичное отклонение: 0.5047771785649585
Коэффициент детерминации: -0.7850168571303473

Результаты кросс-валидации при cv=10

Оптимизация метрики accuracy: 0.8210999999999998
Лучший параметр: {'n_neighbors': 37}
Метрики на тестовом наборе
Среднее квадратичное отклонение: 0.4150903516103452
Коэффициент детерминации: -0.20705810236875521

Оптимизация метрики recall: 0.2507014276846679
Лучший параметр: {'n_neighbors': 1}
Ме

In [28]:
best_k = 1
y_pred_best3 = KNeighborsClassifier(n_neighbors=best_k).fit(x_train, y_train).predict(x_test)

## 7) Сравнение исходной и оптимальной моделей

In [31]:
print('Исходная модель\n')
print_metrics(y_test, y_pred_base)
print('_______________________')
print('\nОптимальная модель\n')
print_metrics(y_test, y_pred_best3)

Исходная модель

Среднее квадратичное отклонение: 0.41988093550434036
Коэффициент детерминации: -0.23508034502386277
_______________________

Оптимальная модель

Среднее квадратичное отклонение: 0.5047771785649585
Коэффициент детерминации: -0.7850168571303473
